In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
rand_state=1000

In [ ]:
df = pd.read_csv('nyc-rolling-sales.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'nyc-rolling-sales.csv'

In [ ]:
df.info()

In [ ]:
df.nunique()

Unnamed won't tell us anything in our analysis, it looks like just an index column.

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

Constructing our variable types: SALE DATE as date, then our numerics and categoricals.

In [ ]:
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'], errors='coerce')

In [ ]:
numerics = ["RESIDENTIAL UNITS", "COMMERCIAL UNITS", "TOTAL UNITS", "LAND SQUARE FEET", "GROSS SQUARE FEET", "SALE PRICE"]
categoricals = ["BOROUGH", "NEIGHBORHOOD", "BUILDING CLASS CATEGORY", "TAX CLASS AT PRESENT", "BUILDING CLASS AT PRESENT",
                "ZIP CODE", "YEAR BUILT", "TAX CLASS AT TIME OF SALE", "BUILDING CLASS AT TIME OF SALE"]

df[numerics]= df[numerics].apply(lambda x: pd.to_numeric(x, errors="coerce"))
df[categoricals]= df[categoricals].apply(lambda x: x.astype("category"))

Being able to look at the sale year and month could also be useful in our analysis, so we'll break out those columns as well.

In [ ]:
df['sale_year'] = pd.DatetimeIndex(df['SALE DATE']).year.astype("category")
df['sale_month'] = pd.DatetimeIndex(df['SALE DATE']).month.astype("category")
pd.crosstab(df['sale_month'],df['sale_year'])

In [ ]:
df.info()

Now let's look for missing values:

In [ ]:
df.isna().sum()

In [ ]:
df.replace(to_replace=' ', value=np.nan, inplace=True)
df.isna().sum() / len(df) * 100

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='Greens')

Easement is entirely blank, apartment number is around 77% blank, so let's drop those columns.

In [ ]:
df.drop(['EASE-MENT', 'APARTMENT NUMBER'], axis=1, inplace=True)

In [ ]:
df=df.dropna()

In [ ]:
sum(df.duplicated())

In [ ]:
# Quite a few duplicates, we'll drop those, too.
df.drop_duplicates(inplace=True)

Now let's look closer at our other missing data:

In [ ]:
# Change all variables to numberic and look at the statistics
temp = df.copy()
for cols in temp.columns:
    temp[cols]=pd.to_numeric(temp[cols], errors='coerce')

temp.info()

In [ ]:
temp.describe().T

Looking at sale price first, because the data seems very skewed and has $0 values, which doesn't make any sense in terms of our analysis.

In [ ]:
df[(df['SALE PRICE']<10000) | (df['SALE PRICE']>10000000)]['SALE PRICE'].count() /len(df)

In [ ]:
df2= df[(df['SALE PRICE']>10000) & (df['SALE PRICE']<10000000)].copy()
df2['SALE PRICE'].describe()

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(df2['SALE PRICE'], kde=True, bins=50, rug=True)
plt.show()

This plot looks better, but it's still quite skewed.

In [ ]:
df2= df2[(df2['SALE PRICE']<4000000)]
plt.figure(figsize=(12,6))
sns.distplot(df2['SALE PRICE'], kde=True, bins=50, rug=True)
plt.show()

That looks much more normal. It's still right skewed, but that makes sense for real estate prices because of luxury homes.

Now let's look at Year Built:

In [ ]:
# Number of 0's
df2[df2['YEAR BUILT']==0]['YEAR BUILT'].count()

In [ ]:
df3=df2[df2['YEAR BUILT']!=0].copy()
sns.distplot(df3['YEAR BUILT'], bins=50, rug=True)
plt.show()

This plot looks good. After removing zeroes, there are no major outliers or nonsensical data.

Now units:

In [ ]:
df3[df3['TOTAL UNITS']==0]['TOTAL UNITS'].count()

In [ ]:
df4=df3[df3['TOTAL UNITS']!=0].copy()
sns.distplot(df4['TOTAL UNITS'], bins=50, rug=True)
plt.show()

Most total units are on the low end, but we have a few outliers.

In [ ]:
df4.describe().T

Better! We filtered out those nonsensical values.

In [ ]:
df4.info()

In [ ]:
# We likely don't need the details of an address; we'll get more information from the neighborhood and borough
df4.drop(['BLOCK','LOT','ADDRESS'], axis=1, inplace=True)

In [ ]:
# Make the borough category more readable
df4['BOROUGH']= df4['BOROUGH'].map({1:'Manhattan', 2:'Bronx', 3: 'Brooklyn', 4:'Queens',5:'Staten Island'})
df4.head()

Based on what I know about NYC real estate, I would guess that borough likely has an effect on sales price.

In [ ]:
df_bar =df4[['BOROUGH', 'SALE PRICE']].groupby(by='BOROUGH').mean().sort_values(by='SALE PRICE', ascending=True).reset_index()
df_bar

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(y = 'BOROUGH', x = 'SALE PRICE', data = df_bar, palette="Dark2" )
plt.title('Average SALE PRICE on each BOROUGH')
plt.show()

That seems like a correct assumption, because the average sales price varies quite a bit between boroughs. Manhattan is clearly the most expensive, with Staten Island being the cheapest borough.

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y = 'BOROUGH', x = 'SALE PRICE', data = df4, palette="Set1" )
plt.title('Box plots for SALE PRICE on each BOROUGH')
plt.show()

Manhattan sales prices are normally distributed with a wide range, so although it's the most expensive borough, buyers may still be able to find cheaper options. The Bronx and Staten Island have a similar range, but the Bronx has more higher priced outliers. Brooklyn is also a more affordable borough but is right skewed, which tells us that there are still quite a few higher priced sales. Queens is similar to Brooklyn, but less skewed and with a lower range, so it would be more affordable.

Now let's look at number of sales per month.

In [ ]:
df_bar=df4[['sale_month', 'SALE PRICE']].groupby(by='sale_month').count().sort_values(by='sale_month', ascending=True).reset_index()
df_bar.columns.values[1]='Sales_count'
df_bar

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(y = 'sale_month', x = 'Sales_count', data = df_bar, palette="Set2" )
plt.title('count SALEs by each month')
plt.show()

There's not a clear pattern here. The most popular months to buy/sell are March, June, September, and December, which are spaced relatively evenly throughout the yeat. The least popular month is August, which is surprising to me because I thought families would try to move before school.

In [ ]:
df_bar2 =df4[['sale_month', 'SALE PRICE']].groupby(by='sale_month').mean().sort_values(by='SALE PRICE', ascending=True).reset_index()
df_bar

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(y = 'sale_month', x = 'SALE PRICE', data = df_bar2, palette="Dark2" )
plt.title('Average SALE PRICE on each MONTH')
plt.show()

I was curious about the relationship between price and sale month, so I built this graph out as well. It seems relatively even -- I don't see a huge pattern here. It is interesting, though, that the month with the lowest sales, August, also has the highest average sales price; that could be related.